In [1]:
from __future__ import division
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2"

In [2]:
from tensorflow.keras.models import load_model
model = load_model('/data/dfc13/DoublePulse/Models/TransferTraining.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 9218      
Total params: 14,723,906
Trainable params: 14,723,906
Non-trainable params: 0
_________________________________________________________________


In [12]:
data_tc = np.load('/fastio2/dasha/double_pulse/data_NuTauCC.npy', mmap_mode='r')
data_tn = np.load('/fastio2/dasha/double_pulse/data_NuTauNC.npy', mmap_mode='r')
data_mc = np.load('/fastio2/dasha/double_pulse/data_NuMuCC.npy', mmap_mode='r')
data_mn = np.load('/fastio2/dasha/double_pulse/data_NuMuNC.npy', mmap_mode='r')
data_ec = np.load('/fastio2/dasha/double_pulse/data_NuECC.npy', mmap_mode='r')
data_en = np.load('/fastio2/dasha/double_pulse/data_NuENC.npy', mmap_mode='r')
data_eg = np.load('/fastio2/dasha/double_pulse/data_NuEGR.npy', mmap_mode='r')

In [13]:
print(data_tc.shape,data_ec.shape,data_mc.shape,data_tn.shape,data_en.shape,data_mn.shape,data_eg.shape)

(1486806, 1) (1523801, 1) (1652927, 1) (283320, 1) (755078, 1) (564278, 1) (49617, 1)


In [5]:
#VGG 
mean = 0.0016132531 
std = 0.016890975
#other
#mean = 0.0016005471 
#std = 0.016772727
print(mean, std)

0.0016132531 0.016890975


In [ ]:
images = data_ec['image'][:, 0]
images = (images - mean)/std
tensors = tf.convert_to_tensor(images.reshape(images.shape[0],300,60,1))
rgbimages = tf.image.grayscale_to_rgb(tensors)
preds = model.predict(rgbimages)

In [15]:
preds = []
for n,im in enumerate(data_ec['image']):    #mean = np.mean(im)
    #std = np.std(im)
    im = (im-mean)/std
    im = im.reshape((1,300,60,1))
    image = tf.convert_to_tensor(im.reshape(1,300,60,1))
    im = tf.image.grayscale_to_rgb(image)
    pred = model.predict(im)
    preds.append(pred)
    print("\rPercent = "+str(n/data_ec.shape[0]*100)+" "+str(n)+" of "+str(data_ec.shape[0]), end="")
preds = np.array(preds)
print(len(preds),preds[0])

Percent = 99.9999343746329 1523800 of 152380111523801 [[0.10322284 0.89677715]]


In [11]:
#preds= preds.reshape(data_ec.shape[0],2)
print(len(preds),preds)

1633438 [[0.52415895 0.47584108]
 [0.7686135  0.23138645]
 [0.23371112 0.7662889 ]
 ...
 [0.5527761  0.4472239 ]
 [0.7706971  0.22930284]
 [0.45121384 0.54878616]]


In [16]:
np.save('/fastio2/dasha/double_pulse/preds_NuECC_TransferLearning_200k.npy', preds)

In [17]:
preds = np.load('/fastio2/dasha/double_pulse/preds_NuECC_TransferLearning_200k.npy')
print(preds[0:10])

[[[0.10322284 0.89677715]]

 [[0.23587833 0.76412165]]

 [[0.44646564 0.5535344 ]]

 [[0.54788595 0.45211414]]

 [[0.2982127  0.7017873 ]]

 [[0.7736356  0.22636431]]

 [[0.5152664  0.48473367]]

 [[0.33409092 0.6659092 ]]

 [[0.5671227  0.43287733]]

 [[0.21103147 0.78896856]]]


In [18]:
print(preds.shape)

(1523801, 1, 2)
